# Project objective
This project is designed to review naive Bayes method and its python implementation using cancer cell line encyclopedia dataset.

Information about the dataset, some technical details about the used machine learning method(s) and mathematical details of the quantifications approaches are provided in the code. 

# Packages we work with in this notebook
We are going to use the following libraries and packages:

* **numpy**: NumPy is the fundamental package for scientific computing with Python. (http://www.numpy.org/)
* **sklearn**: Scikit-learn is a machine learning library for Python programming language. (https://scikit-learn.org/stable/)
* **pandas**: Pandas provides easy-to-use data structures and data analysis tools for Python. (https://pandas.pydata.org/)


In [0]:
import numpy as np
import pandas as pd
import sklearn as sk

# Introduction to the dataset

**Name**: Cancer Cell Line Encyclopedia dataset

**Summary**: Identifying tissue of origin of cancer cell lines using their gene expression. Cell lines from 6 tissues were chosen for this code including: breast, central_nervous_system, haematopoietic_and_lymphoid_tissue, large_intestine, lung, skin

**number of features**: 500 (real, positive) 
Top 500 genex based on variance of their expression in the dataset is chosen. The right way to select the features is to do it only on the training set to eliminate information leak from test set. But to simplify the process for the sake of this teaching code, we use all the dataset.

**Number of data points (instances)**: 550

**dataset accessibility**: Dataset is available as part of PharmacoGx R package (https://www.bioconductor.org/packages/release/bioc/html/PharmacoGx.html)

**Link to the dataset**: https://portals.broadinstitute.org/ccle




## Importing the dataset
We can import the dataset in multiple ways

**Colab Notebook**: You can download the dataset file (or files) from the link (if provided) and uploading it to your google drive and then you can import the file (or files) as follows:

**Note.** When you run the following cell, it tries to connect the colab with google derive. Follow steps 1 to 5 in this link (https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/) to complete the 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

# This path is common for everybody
# This is the path to your google drive
input_path = '/content/gdrive/My Drive/'
# reading the data (target)
target_dataset_features = pd.read_csv(input_path + 'CCLE_ExpMat_Top500Genes.csv', index_col=0)
target_dataset_output = pd.read_csv(input_path + 'CCLE_ExpMat_Phenotype.csv', index_col=0)
# Transposing the dataframe to put features in the dataframe columns
target_dataset_features = target_dataset_features.transpose()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Local directory**: In case you save the data in your local directory, you need to change "input_path" to the local directory you saved the file (or files) in.

**GitHub**: If you use my GitHub (or your own GitHub) repo, you need to change the "input_path" to where the file (or files) exist in the repo. For example, when I clone ***ml_projects*** from my GitHub, I need to change "input_path" to 'data/' as the file (or files) is saved in the data dicretory in this repository. 

**Note.**: You can also clone my ***ml_projects*** repository (here: https://github.com/alimadani/ml_projects) and follow the same process.

## Making sure about the dataset characteristics (number of data points and features)

In [3]:
print('number of features: {}'.format(target_dataset_features.shape[1]))
print('number of data points: {}'.format(target_dataset_features.shape[0]))

number of features: 500
number of data points: 550


## Data preparation
We need to prepare the dataset for machine learnign modeling. Here we prepare the data in 2 steps:

1) Selecting target columns from the output dataframe (target_dataset_output)

2) Converting tissue names to integers (one for each tissue)

In [0]:
# tissueid is the column that contains tissue type information
output_var_names = target_dataset_output['tissueid']
# converting tissue names to labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(output_var_names)
output_var = le.transform(output_var_names)

# we would like to use all the features as input features of the model
input_features = target_dataset_features

## Splitting data to training and testing sets

We need to split the data to train and test, if we do not have a separate dataset for validation and/or testing, to make sure about generalizability of the model we train.

**test_size**: Traditionally, 30%-40% of the dataset cna be used for test set. If you split the data to train, validation and test, you can use 60%, 20% and 20% of teh dataset, respectively.

**Note.**: We need the validation and test sets to be big enough for checking genralizability of our model. At the same time we would like to have as much data as possible in the training set to train a better model.

**random_state** as the name suggests, is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices in your case.


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_features, output_var, test_size=0.30, random_state=5)

## Building the supervised learning model
We want to build a multi-class classification model as the output variable include multiple classes. Here we build a naive Bayes model.

### Naive Bayes
To understand naive Bayes algotirhm, we need to know what Bayes theorem. Bayes theorem related conditional rpobabilities as follows:

\begin{equation*} p(A|B)p(B)=p(B|A)p(A) \end{equation*}
that can be rewritten as

\begin{equation*} p(A|B)=\frac{p(B|A)p(A)}{p(B)} \end{equation*}

where p(A) and p(B) are probabilities of events A and B, respectively. p(A|B) and p(B|A) are also conditional probabilities of A given B and B given A, respectively.

**Example without numbers**

Now let's assume we have 3 features X1, X2 and X3 and we want to identify the probability of class C for sample A with feature values *x1*, *x2* and *x3*:

\begin{equation*} p(class=C|X1=x1, X2=x2 , X3=x3)=\frac{p(X1=x1|class=C)p(X2=x2|lass=C)p(X3=x3|class=C)p(class=C)}{p(X1=x1)p(X2=x2)p(X3=x3)} \end{equation*}

where 
\begin{equation*} p(X1=x1, X2=x2 , X3=x3)=p(X1=x1)p(X2=x2)p(X3=x3) \end{equation*}
and
\begin{equation*} p(X1=x1, X2=x2 , X3=x3|class=C)=p(X1=x1|class=C)p(X2=x2|class=C)p(X3=x3|lass=C)p(class=C) \end{equation*}

as the features are independent variables. 

**Real life example with numbers**

We want to know the chance of having breast cancer if the diagnosis test is positive for a woman with the age between 40 and 60. This example is mainly for understanding Bayes theorem not Naive Bayes classifier. In case of naive Bayes algorithm, this process can be easily extended to multiple features as described in the above example.

***Assumptions (not necessarily correct)***

* 2% of women between 40 and 60 have breast cancer
* True positive rate is 95% (if a woman has breast cancer, it will be diagnosed with 95% probability). Therefore, 5% of the time the women without breast cancer will be diagnosed positively by the test.

Now the question is *What is the chance of havign breast cancer if a woman has positive result from a diagnosis test?*

\begin{equation*} p(having \quad breast \quad cancer|positive)=\frac{p(positive|breast \quad cancer)p(breast cancer)}{p(positive)} \end{equation*}

where 


\begin{equation*} p(positive) = p(positive|having \quad breast \quad cancer)p(having \quad breast \quad cancer) \\+ p(positive|not \quad having \quad breast \quad cancer)p(not \quad having \quad breast \quad cancer)\\=
0.95*0.02+0.05*0.98\\=0.068\end{equation*}

Therefore,

\begin{equation*} p(having \quad breast  \quad cancer|positive)=\frac{p(positive|breast \quad cancer)p(having \quad breast \quad cancer)}{p(positive)}\\= \frac{0.95*0.02}{0.068}\\=0.28\end{equation*}


As we can see, there is only 28% chance of having cancer upon positive test result. Although the numbers were not clinically valid numbers, we deal with similar results in disease diagnosis. This is one of the reasons that further checkups by phycisions are mandatory upon positive results. Do not panic when you have a positive result but follow up with your doctor immediately.

**Note.** Naive Bayes classifier is called ***Naive*** as it assumes each feature will independently contribute in prediction of a class for each data point (sample).

In [6]:
from sklearn.naive_bayes import GaussianNB

# Create naive Bayes object
nb = GaussianNB()

# Train the model using the training sets
nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Prediction of test (or validation) set
We now have to use the trained model to predict y_test.

In [0]:
# Make predictions using the testing set
y_pred = nb.predict(X_test)

## Evaluating performance of the model
We need to assess performance of the model using the predictions of the test set. We use accuracy and balanced accuracy. Here are their definitions:

* **recall** in this context is also referred to as the true positive rate or sensitivity

How many relevant item are selected




$${\displaystyle {\text{recall}}={\frac {tp}{tp+fn}}\,} $$

 

* **specificity** true negative rate



$${\displaystyle {\text{true negative rate}}={\frac {tn}{tn+fp}}\,}$$

* **accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$$ {\displaystyle {\text{accuracy}}={\frac {tp+tn}{tp+tn+fp+fn}}\,}$$


\begin{equation*} accuracy=\frac{number\:of\:correct\:predictions}{(total\:number\:of\:data\:points (samples))} \end{equation*}


* **balanced accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$${\displaystyle {\text{balanced accuracy}}={\frac {recall+specificity
}{2}}\,}$$


In [8]:
from sklearn import metrics

print("accuracy of the predictions:", metrics.accuracy_score(y_test, y_pred))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y_test, y_pred))

accuracy of the predictions: 0.9151515151515152
blanced accuracy of the predictions: 0.9045060422034106
